<!-- @format -->

#### Do all the imports


In [1]:
import requests 
from postgres import Postgres
from dotenv import load_dotenv
from os import getenv
from json import dump

<!-- @format -->

#### load the environments


In [2]:
load_dotenv('techtalk_SEC.secrets')
load_dotenv('techtalk_SEC.env')

True

<!-- @format -->

#### prepare the request


In [3]:

r= requests.Request(method='GET',url=f'{getenv("WEB_URL")}/files/company_tickers.json',headers={
    "User-Agent":"PWTTechTalkBot/1.0 Port Wallis Technologies contact - rwolfe@portwallistechnologies.com",
    "Accept":"application/json"
} )
s = requests.Session()

<!-- @format -->

#### Go get the data


In [4]:

req= s.send( r.prepare())

<!-- @format -->

#### Save the response to an external file (completely optional)


In [5]:
with open(file='cik_tickers.json',mode='w') as file:
    dump(req.json(),file)   

<!-- @format -->

#### Stash the response data in a variable


In [6]:
the_json = req.json()

#### stash the ticker mapping into a dict for later

In [7]:
ticker_cik  = { the_json[a]['ticker']: the_json[a]["cik_str"] for a in the_json}
cik_ticker  = { the_json[a]['cik_str']: the_json[a]["ticker"] for a in the_json}

<!-- @format -->

#### Stash the response in a database table

1. create the connection
1. clear out the table
1. insert the new data


In [8]:
db = Postgres()
    
# always clear out the table first

db.run(f'truncate table "{getenv("PG_SCHEMA")}".company_ticker')

# bad for us to just toss away this but ...

toss_me = [ db.run(  f"""insert  into "{getenv("PG_SCHEMA")}".company_ticker(
                cik_str,
                ticker,
                title) 
               values (
                   '{the_json[a]["cik_str"]}',
                   '{the_json[a]["ticker"]}',
                   '{the_json[a]["title"].replace("'","''")}') """ ) 
       for a in the_json]

del(toss_me)

<!-- @format -->

#### Pull your favorite company's CIK from the database table


In [9]:
ticker_symbol = 'AES'

company_cik :str = db.one(f"""select cik_str from 
                          "{getenv("PG_SCHEMA")}".company_ticker where ticker = '{ticker_symbol}' """)

# CIK has to be 0 padded to 10 digits with leading 0's or it just honks 

company_cik = company_cik.rjust(10,'0')

<!-- @format -->

#### And go get the list of recent submissions for that company


In [10]:
r.url = f"""{getenv("DATA_URL")}/submissions/CIK{company_cik}.json"""

req= s.send( r.prepare())

<!-- @format -->

#### Save the results to a file again


In [11]:
with open(file=f'submissions-{ticker_symbol}.json',mode='w') as file:
    dump(req.json(),file)

<!-- @format -->

#### save the submissions to a couple of dicts for later use


In [12]:

submissions:dict = req.json()

recent_filings:dict = submissions["filings"]["recent"]

submission_count =  len(recent_filings["accessionNumber"])

sub_list = []

for i in range(submission_count):
    sub_list.append( {k: recent_filings[k][i] for k in recent_filings.keys()})



In [13]:
form_names = set()

toss_me = [
    form_names.add(sub_list[i]["form"])
    for i in range(len(sub_list))
    if sub_list[i]["form"] not in form_names
]

all_forms:dict[str:list] = {k:list() for k in form_names}


toss_me = [
    all_forms[sub_list[i]["form"]].append(sub_list[i]) 
    for i in range(submission_count) 
]

del(toss_me)
